<a href="https://colab.research.google.com/github/Ali-mohammadi-design/LLM_Engineering_and_Machine_Learning/blob/main/Design_with_argue_sequential_chain_for_problem_solving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install langchain
!pip install openai
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

In [4]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [5]:
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [37]:
llm= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=1)

In [67]:
human_template1='''Consider the following problem: \n {problem}
\n There are various solutions available to resolve this issue.
Can you please suggest the best solution amongst them?
your answer should be short and clear.
'''
prompt1=ChatPromptTemplate.from_template(human_template1)
chain1=LLMChain(llm=llm, prompt=prompt1, output_key='solution')

In [68]:
human_template2='''
we have faced this problem:\n{problem}
\n somebody has proposed this solution to resolve the problem: \n{solution}
To perform this solution, we face several problems.
Please identify the most critical issue that exists in implementing this solution.
your output should be short and clear.

'''
prompt2=ChatPromptTemplate.from_template(human_template2)
chain2=LLMChain(llm=llm, prompt=prompt2, output_key='new_problem')

In [69]:
human_template3='''
we faced this problem at first:\n{problem}
\n then somebody proposed this solution to resolve the problem: \n{solution}
\n then somebody else mentioned this problem existing in this solution: \n{new_problem}
Please consider the problem existed in the solution and return a problem statement.
The problem statement should be short (in one sentenc), and clear.

'''
prompt3=ChatPromptTemplate.from_template(human_template3)
chain3=LLMChain(llm=llm, prompt=prompt3, output_key='new_statement')

In [71]:
seq_chain=SequentialChain(chains=[chain1,chain2, chain3], input_variables=['problem'],output_variables=['solution','new_problem','new_statement'],verbose=False)


In [75]:
i=0
my_problem=input('\nPlease write down your problem:\n')
while (i==0):
   result=seq_chain(my_problem)
   solution=result['solution']
   new_problem=result['new_problem']
   print(f'The main problem is: \n\n{my_problem}.\n\n we can propose this solution to resolve it:\n\n{solution}. \n\n However this solution has this probelm:\n{new_problem}.')
   answer=input('\n\n\n Do you need help to resolve this probem? Y or N \n')
   if answer=='N':
    new_issue=input('\n\ndo you have new problem to be shared with me? Y or N\n')
    if new_issue=='Y':
      my_problem=input('\n\nPlease write down your problem:\n')
    else:
      i=1
   else:
    my_problem=result['new_statement']


Please write down your problem:
I donot have money
The main problem is: 

I donot have money.

 we can propose this solution to resolve it:

One possible solution could be to find a job or source of income to earn money.. 

 However this solution has this probelm:
The most critical issue in implementing the solution of finding a job or source of income is the lack of available job opportunities or limited access to resources for creating income-generating opportunities..



 Do you need help to resolve this probem? Y or N 
N


do you have new problem to be shared with me? Y or N
Y


Please write down your problem:
I need food
The main problem is: 

I need food.

 we can propose this solution to resolve it:

The best solution would be to go grocery shopping and buy the necessary food items.. 

 However this solution has this probelm:
The most critical issue in implementing this solution is potentially not having enough money to purchase the necessary food items..



 Do you need help t